Plan:
1. Angelas Code in meinen übersetzen
2. aus Angelas Code alle möglichen Mutationen rausschreiben, aber nur einmal
2. --> Liste mit allen vorkommenden Mutationen
3. Funktion schreiben, die für jede dieser (einzeln!) alle Sequenzen raussucht in der diese vorkommt
4. --> ein dataframe, mit mutation als spaltenname und allen sequenzen wo es drin vorkommt als Reihen drunter und noch einer Spalte mit mutcount dahinter, pro Mutation werden zwei Spalten an den dataframe angehängt
5. (dadurch können Reihen spezifisch nach mutcount angeprochen werden)
6. innerhalb einer Mutationsspalte, alle mit dem selben mutcount: fscore vergleich : Varianz der Werte als Maß für impact
-> je unwichtiger die Mutation, desto weniger macht sie was beim fscore, desto weniger sollten sich die Werte unterscheiden, desto kleiner sollte die Varianz sein

In [1]:
import pandas as pd
import numpy as np

In [86]:
# Datensatz einlesen
original_dms_data = pd.read_csv('/Users/liza/Desktop/Bioinfo Project/DMS_data/GFP_AEQVI_Sarkisyan_2016.csv')
# split first column of df into multiple columns
original_dms_data_col = original_dms_data
only_mutants = original_dms_data["mutant"].to_frame()
original_dms_data_col[['m1', 'm2', 'm3', 'm4', 'm5', 'm6', 'm7', 'm8', 'm9', 'm10', 'm11', 'm12', 'm13', 'm14', 'm15']] = original_dms_data_col['mutant'].str.split(':', 15, expand=True)
original_dms_data_col.head()


/var/folders/sk/q8h0c3zs1v9dt93y5m134swr0000gn/T/ipykernel_57463/404050330.py:6: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  original_dms_data_col[['m1', 'm2', 'm3', 'm4', 'm5', 'm6', 'm7', 'm8', 'm9', 'm10', 'm11', 'm12', 'm13', 'm14', 'm15']] = original_dms_data_col['mutant'].str.split(':', 15, expand=True)


,mutant,mutated_sequence,DMS_score,DMS_score_bin,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,m11,m12,m13,m14,m15
0,K3R:V55A:Q94R:A110T:D117G:M153K:D216A,MSRGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKF...,1.301030,0,K3R,V55A,Q94R,A110T,D117G,M153K,D216A,None,None,None,None,None,None,None,None
1,K3Q:V16A:I167T:L195Q,MSQGEELFTGVVPILAELDGDVNGHKFSVSGEGEGDATYGKLTLKF...,3.137350,1,K3Q,V16A,I167T,L195Q,None,None,None,None,None,None,None,None,None,None,None
2,K3Q:Y143C:N164D:S205P:A227T,MSQGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKF...,1.553913,0,K3Q,Y143C,N164D,S205P,A227T,None,None,None,None,None,None,None,None,None,None
3,K3Q:Y143N:V193A,MSQGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKF...,3.404237,1,K3Q,Y143N,V193A,None,None,None,None,None,None,None,None,None,None,None,None
4,K3R,MSRGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKF...,3.738586,1,K3R,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [87]:
# zählen wieviele Mutationen jede Sequenz hat
list_mut_count_in_progress = []
for i in range(len(original_dms_data['mutant'])):
    list_mut_count_in_progress.append(original_dms_data['mutant'].iloc[i].count(':'))
list_mut_count_prae = np.array(list_mut_count_in_progress)
list_mut_count = (list_mut_count_prae + 1)
df_mutation_counts = pd.DataFrame(list_mut_count)

Dataframe erstellen aus den Spalten: (WORKING_DATAFRAME)
- Mutationsanzahl aus df_mutation_counts
- fscore aus original_dms_data_col
- m1 bis b11 aus original_dms_data_col

In [89]:
#zusammenfügen
working_dataframe_prae = pd.concat([original_dms_data_col, df_mutation_counts], axis="columns")
#alle Spalten löschen die stören
working_dataframe = working_dataframe_prae.drop(['mutant', 'mutated_sequence', 'DMS_score_bin'], axis=1)
working_dataframe.rename(columns={working_dataframe.columns[16]: 'mut_count'}, inplace=True)
working_dataframe.head()

,DMS_score,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,m11,m12,m13,m14,m15,mut_count
0,1.301030,K3R,V55A,Q94R,A110T,D117G,M153K,D216A,None,None,None,None,None,None,None,None,7
1,3.137350,K3Q,V16A,I167T,L195Q,None,None,None,None,None,None,None,None,None,None,None,4
2,1.553913,K3Q,Y143C,N164D,S205P,A227T,None,None,None,None,None,None,None,None,None,None,5
3,3.404237,K3Q,Y143N,V193A,None,None,None,None,None,None,None,None,None,None,None,None,3
4,3.738586,K3R,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1


In [91]:
#alle möglichen Mutationen rausschreiben in den Spalten
from pandas import unique
working_dataframe_only_ms = working_dataframe.loc[:, ["m1", "m2", "m3", 'm4', 'm5', 'm6', 'm7', 'm8', 'm9', 'm10', 'm11', 'm12', 'm13', 'm14', 'm15']]
working_dataframe_only_ms


,m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,m11,m12,m13,m14,m15
0,K3R,V55A,Q94R,A110T,D117G,M153K,D216A,None,None,None,None,None,None,None,None
1,K3Q,V16A,I167T,L195Q,None,None,None,None,None,None,None,None,None,None,None
2,K3Q,Y143C,N164D,S205P,A227T,None,None,None,None,None,None,None,None,None,None
3,K3Q,Y143N,V193A,None,None,None,None,None,None,None,None,None,None,None,None
4,K3R,None,None,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51709,Y237N,None,None,None,None,None,None,None,None,None,None,None,None,None,None
51710,Y237C,None,None,None,None,None,None,None,None,None,None,None,None,None,None
51711,Y237D,None,None,None,None,None,None,None,None,None,None,None,None,None,None
51712,Y237F,None,None,None,None,None,None,None,None,None,None,None,None,None,None


1. alle Werte des DataFrames mit values.flatten() in eine eindimensionale Liste umgewandeln
2. tolist() verwendet, um die Liste in eine normale Python-Liste umzuwandeln
3.  Liste in ein Set umgewandeln (nur eindeutige Elemente)
4. set zurück in Liste

In [92]:
all_possible_mutations = working_dataframe_only_ms.values.flatten().tolist()
all_possible_mutations = list(set(all_possible_mutations))
print(all_possible_mutations)

# 1884 mögliche Mutationen in dem Datenset

['E32Q', 'P196T', 'E34G', 'D103E', 'T108N', 'K79V', 'N159D', 'M218R', 'G67N', 'G160V', 'Y237N', 'K156L', 'D36A', 'N23T', 'G67D', 'D19Y', 'H77R', 'R73S', 'I229D', 'Y66I', 'G104V', 'P56T', 'M88V', 'L18P', 'D133Y', 'E90K', 'K162W', 'N149C', 'H77Y', 'S175R', 'N149R', 'L60S', 'N146T', 'H25N', 'N121G', 'K41E', 'I171T', 'G40C', 'K131L', 'I47V', 'I167F', 'I161N', 'Q80H', 'Q184W', 'F84S', 'R215H', 'M153S', 'V193G', 'D103G', 'C48F', 'Y39L', 'L137Q', 'L195V', 'V193L', 'Y200H', 'D234T', 'S86P', 'I171F', 'M233V', 'N121T', 'D102A', 'Y182N', 'Y145C', 'K113V', 'I161G', 'F46C', 'F8T', 'E5D', 'M218S', 'K131V', 'P54H', 'Y151V', 'K113E', 'Y145S', 'I152F', 'V93L', 'K26L', 'D234A', 'I128F', 'E17V', 'F114Y', 'E235G', 'G116D', 'N105I', 'D102N', 'I229V', 'Y39C', 'E5A', 'P56H', 'I171S', 'G67R', 'G40S', 'R168C', 'F114P', 'L220M', 'K26T', 'N159R', 'N105F', 'A226G', 'G127A', 'I128P', 'Y92F', 'P187H', 'E17D', 'K214I', 'H25Q', 'L236Q', 'L53S', 'A227P', 'E213G', 'E235Q', 'I152L', 'N23G', 'D129A', 'Y182C', 'F71L', 'A2

In [ ]:
# funktion schreiben, die zu jeder dieser Mutationen alle Sequenzen raussucht in der diese Mut vorkommt
#-> Frage: für jede einzeln ein dataset oder für alle eins?
how_often_df = pd.DataFrame()
for i in all_possible_mutations:
    for j in range(len(all_possible_mutations)):
        how_often_df.assign(m = (only_mutants.iloc[j].str.contains(i)))
print(how_often_df)


test: aus dem dataframe mit den mutantennamen raussuchen, ob in einer bestimmten Zeile die bestimmte Mutation vorkommt

In [77]:
only_mutants.iloc[3].str.contains("V193A")

mutant    True
Name: 3, dtype: bool

In [75]:
how_often_df.assign(i = working_dataframe_only_ms.apply(lambda x: x.str.contains(i), axis=1))

pandas.core.frame.DataFrame